In [1]:
import glob
import torch

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import random
from PIL import Image
from pathlib import Path

# from sklearn.preprocessing import MinMaxScaler
# from tensorflow.keras.utils import to_categorical
# import imageio   # Convert to an image 
import torch.optim as optim
import torch.nn.functional as F

import torchvision
from torchvision import *
from torch.utils.data import Dataset, DataLoader

In [2]:
print("The current DIR: ", os.getcwd())
np.random.seed(1000)
torch.manual_seed(1000)

The current DIR:  /home/shsingh/knowledge_distillation/notebooks


In [14]:
DATASET_DIR =Path( '/home/shsingh/knowledge_distillation/knowledge_distillation/dataset/scratch')
dataset_name= 'MICCAI_BraTS20_trainigdata_npz'
# meta_csv = 'name_mapping.csv'

dataset_dir = DATASET_DIR / dataset_name
naming_csv_path ='/home/shsingh/knowledge_distillation/knowledge_distillation/dataset/scratch/MICCAI_BraTS2020_Data/name_mapping.csv'
df_name =  pd.read_csv(naming_csv_path)
df_name =  df_name.drop(['BraTS_2017_subject_ID','BraTS_2018_subject_ID','TCGA_TCIA_subject_ID','BraTS_2019_subject_ID'], axis =1)
df_name.head()

,Grade,BraTS_2020_subject_ID
0,HGG,BraTS20_Training_001
1,HGG,BraTS20_Training_002
2,HGG,BraTS20_Training_003
3,HGG,BraTS20_Training_004
4,HGG,BraTS20_Training_005


In [15]:
survival_csv = '/home/shsingh/knowledge_distillation/knowledge_distillation/dataset/scratch/MICCAI_BraTS2020_Data/survival_info.csv'
survival_df =  pd.read_csv(survival_csv)
survival_df.head()

,Brats20ID,Age,Survival_days,Extent_of_Resection
0,BraTS20_Training_001,60.463,289,GTR
1,BraTS20_Training_002,52.263,616,GTR
2,BraTS20_Training_003,54.301,464,GTR
3,BraTS20_Training_004,39.068,788,GTR
4,BraTS20_Training_005,68.493,465,GTR


In [16]:
def get_mod_label(subject, seg, mod, df_name):
    label = ""       
    #Check if HEALTHY. 
    #Healthy:if the seg != [2,3] i.e. non-enhancing tumour and nunique_modularity != 0 assign label == Healthy
    seg_unique = np.unique(seg)
    mod_unique = np.unique(mod)
    mod_sum = np.sum(mod)
    enhancing_seg_lbl = [2,3] #TODO: take this label as user input:: enhancing_seg_lbl
    non_enhancing_seg_lbl = [0,1] #TODO: take this label as user input::non_enhancing_seg_lbl
    if (mod_sum!=0 and (all(i not in  seg_unique for i in enhancing_seg_lbl))):
        return 'healthy'
    elif(mod_sum!=0 and (any(i in seg_unique for i in enhancing_seg_lbl))):
        label = df_name.loc[df_name['BraTS_2020_subject_ID']== subject]['Grade'].values[0]
        return label
         
        
def get_all_mod_label(subject,seg_path, flair_path, t1_path, t1ce_path, t2_path, df_name):
    seg = np.load(seg_path)
    flair = np.load(flair_path)
    t1 = np.load(t1_path)
    t1ce = np.load(t1ce_path)
    t2 = np.load(t2_path)
    
    mod_list = [flair, t1, t1ce, t2]
    lbl_discard = 'discard'
    #Check if only  0's mod and seg
    if (np.sum(mod_list)) == 0: 
        return lbl_discard, lbl_discard, lbl_discard, lbl_discard
    else:
        flair_lbl = get_mod_label(subject, seg, flair, df_name)
        t1_lbl = get_mod_label(subject, seg, t1, df_name)
        t1ce_lbl = get_mod_label(subject, seg, t1ce, df_name)
        t2_lbl = get_mod_label(subject, seg, t2, df_name)
    return flair_lbl, t1_lbl, t1ce_lbl, t2_lbl

def create_mapping_csv(dataset_dir, df_name):
    map_df = pd.DataFrame(columns=['subject_id','image_idx','image_path','label_flair','label_t1', 'label_t1ce', 'label_t2'])
    subject_ids = sorted(os.listdir(dataset_dir))
    substring_subject_id ='BraTS20_'
    for subject in subject_ids:
        if substring_subject_id in subject:
            subject_path = dataset_dir / subject
            subject_modularities = sorted(os.listdir(subject_path))
            for modularity in subject_modularities:
                if '_seg' in modularity:
                    subject_seg_dir = subject_path / modularity
                    subject_seg_files = sorted(os.listdir(subject_seg_dir))
                    for file in subject_seg_files:
                        subject_seg_path = subject_seg_dir / file
                        image_id = str(file).split(".")[0]
                        subject_seg_path_str = str(subject_seg_path)
                        subject_flair_path = subject_seg_path_str.replace('seg', 'flair')
                        subject_t1_path = subject_seg_path_str.replace('seg', 't1')
                        subject_t1ce_path = subject_seg_path_str.replace('seg', 't1ce') 
                        subject_t2_path = subject_seg_path_str.replace('seg', 't2')
                        lbl_falir, lbl_t1, lbl_tce, lbl_t2 = get_all_mod_label(subject, subject_seg_path,subject_flair_path,
                                                                               subject_t1_path,subject_t1ce_path,
                                                                               subject_t2_path, df_name)
                        df_tmp = pd.DataFrame([(subject, image_id, subject_seg_path,lbl_falir, lbl_t1, lbl_tce, lbl_t2 )], 
                                          columns=['subject_id','image_idx','image_path','label_flair','label_t1', 'label_t1ce', 'label_t2'])
#                         map_df = map_df.append(df_tmp) 
                        map_df = pd.concat([map_df, df_tmp], axis =0) 
    map_df_reset = map_df.copy()
    map_df_reset = map_df_reset.reset_index(drop=True)
    return map_df, map_df_reset
                    
                
            
    

In [17]:
map_df, map_df_reset = create_mapping_csv(dataset_dir, df_name)


In [18]:
map_df_reset.head()

,subject_id,image_idx,image_path,label_flair,label_t1,label_t1ce,label_t2
0,BraTS20_Training_001,BraTS20_Training_001_seg_0,/home/shsingh/knowledge_distillation/knowledge...,discard,discard,discard,discard
1,BraTS20_Training_001,BraTS20_Training_001_seg_1,/home/shsingh/knowledge_distillation/knowledge...,discard,discard,discard,discard
2,BraTS20_Training_001,BraTS20_Training_001_seg_10,/home/shsingh/knowledge_distillation/knowledge...,healthy,healthy,healthy,healthy
3,BraTS20_Training_001,BraTS20_Training_001_seg_100,/home/shsingh/knowledge_distillation/knowledge...,HGG,HGG,HGG,HGG
4,BraTS20_Training_001,BraTS20_Training_001_seg_101,/home/shsingh/knowledge_distillation/knowledge...,HGG,HGG,HGG,HGG


In [19]:
map_df_reset.to_csv('/home/shsingh/knowledge_distillation/knowledge_distillation/dataset/scratch/meta_data_mapping.csv',index=False)

In [20]:
mm = pd.read_csv('/home/shsingh/knowledge_distillation/knowledge_distillation/dataset/scratch/meta_data_mapping.csv')
mm.head()

,subject_id,image_idx,image_path,label_flair,label_t1,label_t1ce,label_t2
0,BraTS20_Training_001,BraTS20_Training_001_seg_0,/home/shsingh/knowledge_distillation/knowledge...,discard,discard,discard,discard
1,BraTS20_Training_001,BraTS20_Training_001_seg_1,/home/shsingh/knowledge_distillation/knowledge...,discard,discard,discard,discard
2,BraTS20_Training_001,BraTS20_Training_001_seg_10,/home/shsingh/knowledge_distillation/knowledge...,healthy,healthy,healthy,healthy
3,BraTS20_Training_001,BraTS20_Training_001_seg_100,/home/shsingh/knowledge_distillation/knowledge...,HGG,HGG,HGG,HGG
4,BraTS20_Training_001,BraTS20_Training_001_seg_101,/home/shsingh/knowledge_distillation/knowledge...,HGG,HGG,HGG,HGG


In [23]:
all_data = mm.merge(survival_df,left_on='subject_id', right_on='Brats20ID', how='left')
all_data.head()

,subject_id,image_idx,image_path,label_flair,label_t1,label_t1ce,label_t2,Brats20ID,Age,Survival_days,Extent_of_Resection
0,BraTS20_Training_001,BraTS20_Training_001_seg_0,/home/shsingh/knowledge_distillation/knowledge...,discard,discard,discard,discard,BraTS20_Training_001,60.463,289,GTR
1,BraTS20_Training_001,BraTS20_Training_001_seg_1,/home/shsingh/knowledge_distillation/knowledge...,discard,discard,discard,discard,BraTS20_Training_001,60.463,289,GTR
2,BraTS20_Training_001,BraTS20_Training_001_seg_10,/home/shsingh/knowledge_distillation/knowledge...,healthy,healthy,healthy,healthy,BraTS20_Training_001,60.463,289,GTR
3,BraTS20_Training_001,BraTS20_Training_001_seg_100,/home/shsingh/knowledge_distillation/knowledge...,HGG,HGG,HGG,HGG,BraTS20_Training_001,60.463,289,GTR
4,BraTS20_Training_001,BraTS20_Training_001_seg_101,/home/shsingh/knowledge_distillation/knowledge...,HGG,HGG,HGG,HGG,BraTS20_Training_001,60.463,289,GTR


In [28]:
all_data.to_csv('/home/shsingh/knowledge_distillation/knowledge_distillation/dataset/scratch/meta_data_survival_mapping.csv',index=False)

In [33]:
all_data = pd.read_csv('/home/shsingh/knowledge_distillation/knowledge_distillation/dataset/scratch/meta_data_survival_mapping.csv')
all_data

,subject_id,image_idx,image_path,label_flair,label_t1,label_t1ce,label_t2,Brats20ID,Age,Survival_days,Extent_of_Resection
0,BraTS20_Training_001,BraTS20_Training_001_seg_0,/home/shsingh/knowledge_distillation/knowledge...,discard,discard,discard,discard,BraTS20_Training_001,60.463,289,GTR
1,BraTS20_Training_001,BraTS20_Training_001_seg_1,/home/shsingh/knowledge_distillation/knowledge...,discard,discard,discard,discard,BraTS20_Training_001,60.463,289,GTR
2,BraTS20_Training_001,BraTS20_Training_001_seg_10,/home/shsingh/knowledge_distillation/knowledge...,healthy,healthy,healthy,healthy,BraTS20_Training_001,60.463,289,GTR
3,BraTS20_Training_001,BraTS20_Training_001_seg_100,/home/shsingh/knowledge_distillation/knowledge...,HGG,HGG,HGG,HGG,BraTS20_Training_001,60.463,289,GTR
4,BraTS20_Training_001,BraTS20_Training_001_seg_101,/home/shsingh/knowledge_distillation/knowledge...,HGG,HGG,HGG,HGG,BraTS20_Training_001,60.463,289,GTR
...,...,...,...,...,...,...,...,...,...,...,...
57190,BraTS20_Training_369,BraTS20_Training_369_seg_95,/home/shsingh/knowledge_distillation/knowledge...,HGG,HGG,HGG,HGG,BraTS20_Training_369,50.000,145,GTR
57191,BraTS20_Training_369,BraTS20_Training_369_seg_96,/home/shsingh/knowledge_distillation/knowledge...,HGG,HGG,HGG,HGG,BraTS20_Training_369,50.000,145,GTR
57192,BraTS20_Training_369,BraTS20_Training_369_seg_97,/home/shsingh/knowledge_distillation/knowledge...,HGG,HGG,HGG,HGG,BraTS20_Training_369,50.000,145,GTR
57193,BraTS20_Training_369,BraTS20_Training_369_seg_98,/home/shsingh/knowledge_distillation/knowledge...,HGG,HGG,HGG,HGG,BraTS20_Training_369,50.000,145,GTR


In [35]:
all_data['image_path'][0]

'/home/shsingh/knowledge_distillation/knowledge_distillation/dataset/scratch/MICCAI_BraTS20_trainigdata_npz/BraTS20_Training_001/BraTS20_Training_001_seg/BraTS20_Training_001_seg_10.npy'

In [21]:
def class_to_idx(label):
    idx = 0
    if label == 'healthy':idx =0
    if label == 'HGG':idx =1
    if label=='LGG': idx = 2        
       
    return idx


class CustomImageDataset_from_csv(Dataset):
    def __init__(self, data_df ,  transform = None , label_transform =  None):
        self.dataframe = data_df #pd.read_csv(annotations_file)
        self.transform = transform
        self.label_transform = label_transform
        
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self , idx):
        img_path =  self.dataframe.loc[idx,'image_path']
#         image = Image.open(img_path)  
        image = np.load(img_path)
        
#         label = self.dataframe.loc[idx, 'label_flair']
        label = self.dataframe.loc[idx, 'label_t1']
        label = class_to_idx(label)
        label = torch.tensor([label])
                             
        
        if self.transform is not None:
            image = self.transform(image)
        if self.label_transform:
            label = self.target_transform(label)
            
        print('Image shape : ',np.shape(image))
        print('label shape : ',np.shape(label))
        
        return(image, label )

In [22]:
image_size = 240 #TODO: define mean, std for each modularitiy expilicity by EDA (as they have differen distribution)

In [23]:
train_data = CustomImageDataset_from_csv(all_data )

train_data = CustomImageDataset_from_csv(all_data ,
           transform=transforms.Compose([
#                                    transforms.Resize((image_size, image_size)), 
                                   transforms.ToTensor()
#                                    transforms.Normalize((0.5, ), (0.5, ))
                               ]))

batch_size = 64
train_dataloader = DataLoader(train_data, batch_size = batch_size , shuffle = True)

arr_li =[]
lbl_li =[]

for images, labels in train_dataloader:
    arr_li.append(images)
    lbl_li.apppend(labels)
    
    
#     fig, ax = plt.subplots(figsize = (20, 20))
#     print(np.shape(images))
#     ax.set_xticks([])
#     ax.set_yticks([])
#     ax.imshow(make_grid((Image.fromarray(images), 10).permute(1,2,0)))
#     ax.set_title((labels))
#     break
    
    
# print(np.shape(labels[0]))

/home/shsingh/knowledge_distillation/knowledge_distillation/dataset/scratch/MICCAI_BraTS20_trainigdata_npz/BraTS20_Training_284/BraTS20_Training_284_seg/BraTS20_Training_284_seg_128.npy
Image shape :  torch.Size([1, 240, 240])
label shape :  torch.Size([1])
/home/shsingh/knowledge_distillation/knowledge_distillation/dataset/scratch/MICCAI_BraTS20_trainigdata_npz/BraTS20_Training_251/BraTS20_Training_251_seg/BraTS20_Training_251_seg_115.npy
Image shape :  torch.Size([1, 240, 240])
label shape :  torch.Size([1])
/home/shsingh/knowledge_distillation/knowledge_distillation/dataset/scratch/MICCAI_BraTS20_trainigdata_npz/BraTS20_Training_291/BraTS20_Training_291_seg/BraTS20_Training_291_seg_19.npy
Image shape :  torch.Size([1, 240, 240])
label shape :  torch.Size([1])
/home/shsingh/knowledge_distillation/knowledge_distillation/dataset/scratch/MICCAI_BraTS20_trainigdata_npz/BraTS20_Training_286/BraTS20_Training_286_seg/BraTS20_Training_286_seg_104.npy
Image shape :  torch.Size([1, 240, 240])


AttributeError: 'list' object has no attribute 'apppend'

In [16]:
labels

tensor([[0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [2],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [2],
        [0],
        [2],
        [0],
        [0]])

In [24]:
labels

tensor([[2],
        [0],
        [0],
        [2],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [2],
        [2],
        [0],
        [2],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [1],
        [2],
        [0],
        [0],
        [2],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [2],
        [0],
        [2],
        [1],
        [0],
        [0],
        [2],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [2],
        [1],
        [0],
        [1],
        [1],
        [0]])

In [18]:
len(arr_li)

1

In [20]:
np.shape(arr_li[0])

torch.Size([64, 1, 240, 240])

# DRAFT CODE

In [7]:
subject_ids = sorted(os.listdir(dataset_dir))
substring_subject_id ='BraTS20_'
for subject in subject_ids:
    if substring_subject_id in subject:
        print(subject)
        subject_path = dataset_dir / subject
        subject_modularities = sorted(os.listdir(subject_path))
        for modularity in subject_modularities:
            if '_seg' in modularity:
                print(modularity)
                subject_seg_dir = subject_path / modularity
                subject_seg_files = sorted(os.listdir(subject_seg_dir))
                for file in subject_seg_files:
#                     print(file)
                    subject_seg_path = subject_seg_dir / file
#                     print(subject_seg_path)
#                     image_id = str(subject_seg_path).split(".")[0]
                    image_id = str(file).split(".")[0]
                    print(image_id)
                    subject_seg_path_str = str(subject_seg_path)
                    subject_flair_path = subject_seg_path_str.replace('seg', 'flair')
                    subject_t1_path = subject_seg_path_str.replace('seg', 't1')
                    subject_t1ce_path = subject_seg_path_str.replace('seg', 't1ce') 
                    subject_t2_path = subject_seg_path_str.replace('seg', 't2')
                    lbl_falir, lbl_t1, lbl_tce, lbl_t2 = get_all_mod_label(subject, subject_seg_path,subject_flair_path,
                                                                               subject_t1_path,subject_t1ce_path,
                                                                               subject_t2_path, df_name)
                    
                    df_tmp = pd.DataFrame([(subject, image_id, subject_seg_path,lbl_falir, lbl_t1, lbl_tce, lbl_t2 )], 
                                          columns=['subject_id','image_idx','image_path','label_flair','label_t1', 'label_t1ce', 'label_t2'])
                    map_df = map_df.append(df_tmp) 

#                     break
                
#         break

BraTS20_Training_001
BraTS20_Training_001_seg
BraTS20_Training_001_seg_0
BraTS20_Training_001_seg_1
BraTS20_Training_001_seg_10
BraTS20_Training_001_seg_100
BraTS20_Training_001_seg_101
BraTS20_Training_001_seg_102
BraTS20_Training_001_seg_103
BraTS20_Training_001_seg_104
BraTS20_Training_001_seg_105
BraTS20_Training_001_seg_106
BraTS20_Training_001_seg_107
BraTS20_Training_001_seg_108
BraTS20_Training_001_seg_109


/tmp/ipykernel_564699/3050114029.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  map_df = map_df.append(df_tmp)
/tmp/ipykernel_564699/3050114029.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  map_df = map_df.append(df_tmp)
/tmp/ipykernel_564699/3050114029.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  map_df = map_df.append(df_tmp)
/tmp/ipykernel_564699/3050114029.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  map_df = map_df.append(df_tmp)
/tmp/ipykernel_564699/3050114029.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  m

BraTS20_Training_001_seg_11
BraTS20_Training_001_seg_110
BraTS20_Training_001_seg_111
BraTS20_Training_001_seg_112
BraTS20_Training_001_seg_113
BraTS20_Training_001_seg_114
BraTS20_Training_001_seg_115
BraTS20_Training_001_seg_116
BraTS20_Training_001_seg_117
BraTS20_Training_001_seg_118
BraTS20_Training_001_seg_119
BraTS20_Training_001_seg_12
BraTS20_Training_001_seg_120


/tmp/ipykernel_564699/3050114029.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  map_df = map_df.append(df_tmp)
/tmp/ipykernel_564699/3050114029.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  map_df = map_df.append(df_tmp)
/tmp/ipykernel_564699/3050114029.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  map_df = map_df.append(df_tmp)
/tmp/ipykernel_564699/3050114029.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  map_df = map_df.append(df_tmp)
/tmp/ipykernel_564699/3050114029.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  m

BraTS20_Training_001_seg_121
BraTS20_Training_001_seg_122
BraTS20_Training_001_seg_123
BraTS20_Training_001_seg_124
BraTS20_Training_001_seg_125
BraTS20_Training_001_seg_126
BraTS20_Training_001_seg_127
BraTS20_Training_001_seg_128
BraTS20_Training_001_seg_129
BraTS20_Training_001_seg_13
BraTS20_Training_001_seg_130
BraTS20_Training_001_seg_131
BraTS20_Training_001_seg_132
BraTS20_Training_001_seg_133
BraTS20_Training_001_seg_134


...

/tmp/ipykernel_564699/3050114029.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  map_df = map_df.append(df_tmp)
/tmp/ipykernel_564699/3050114029.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  map_df = map_df.append(df_tmp)
/tmp/ipykernel_564699/3050114029.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  map_df = map_df.append(df_tmp)
/tmp/ipykernel_564699/3050114029.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  map_df = map_df.append(df_tmp)
/tmp/ipykernel_564699/3050114029.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  m

BraTS20_Training_009_seg_89
BraTS20_Training_009_seg_9
BraTS20_Training_009_seg_90
BraTS20_Training_009_seg_91
BraTS20_Training_009_seg_92
BraTS20_Training_009_seg_93
BraTS20_Training_009_seg_94
BraTS20_Training_009_seg_95
BraTS20_Training_009_seg_96
BraTS20_Training_009_seg_97


/tmp/ipykernel_564699/3050114029.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  map_df = map_df.append(df_tmp)
/tmp/ipykernel_564699/3050114029.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  map_df = map_df.append(df_tmp)
/tmp/ipykernel_564699/3050114029.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  map_df = map_df.append(df_tmp)
/tmp/ipykernel_564699/3050114029.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  map_df = map_df.append(df_tmp)
/tmp/ipykernel_564699/3050114029.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  m

BraTS20_Training_009_seg_98
BraTS20_Training_009_seg_99
BraTS20_Training_010
BraTS20_Training_010_seg
BraTS20_Training_010_seg_0
BraTS20_Training_010_seg_1
BraTS20_Training_010_seg_10
BraTS20_Training_010_seg_100
BraTS20_Training_010_seg_101
BraTS20_Training_010_seg_102
BraTS20_Training_010_seg_103
BraTS20_Training_010_seg_104


KeyboardInterrupt: 

In [8]:
map_df.head()

,subject_id,image_idx,image_path,label_flair,label_t1,label_t1ce,label_t2
0,BraTS20_Training_001,BraTS20_Training_001_seg_0,/home/shsingh/knowledge_distillation/knowledge...,discard,discard,discard,discard
0,BraTS20_Training_001,BraTS20_Training_001_seg_1,/home/shsingh/knowledge_distillation/knowledge...,discard,discard,discard,discard
0,BraTS20_Training_001,BraTS20_Training_001_seg_10,/home/shsingh/knowledge_distillation/knowledge...,healthy,healthy,healthy,healthy
0,BraTS20_Training_001,BraTS20_Training_001_seg_100,/home/shsingh/knowledge_distillation/knowledge...,HGG,HGG,HGG,HGG
0,BraTS20_Training_001,BraTS20_Training_001_seg_101,/home/shsingh/knowledge_distillation/knowledge...,HGG,HGG,HGG,HGG


In [9]:
map_df['image_idx'].value_counts()

BraTS20_Training_001_seg_0      1
BraTS20_Training_007_seg_1      1
BraTS20_Training_007_seg_106    1
BraTS20_Training_007_seg_105    1
BraTS20_Training_007_seg_104    1
                               ..
BraTS20_Training_004_seg_1      1
BraTS20_Training_004_seg_0      1
BraTS20_Training_003_seg_99     1
BraTS20_Training_003_seg_98     1
BraTS20_Training_010_seg_103    1
Name: image_idx, Length: 1402, dtype: int64

In [19]:
x = map_df.copy()
x = x.reset_index(drop=True)
x


,subject_id,image_idx,image_path,label_flair,label_t1,label_t1ce,label_t2
0,BraTS20_Training_001,BraTS20_Training_001_seg_0,/home/shsingh/knowledge_distillation/knowledge...,discard,discard,discard,discard
1,BraTS20_Training_001,BraTS20_Training_001_seg_1,/home/shsingh/knowledge_distillation/knowledge...,discard,discard,discard,discard
2,BraTS20_Training_001,BraTS20_Training_001_seg_10,/home/shsingh/knowledge_distillation/knowledge...,healthy,healthy,healthy,healthy
3,BraTS20_Training_001,BraTS20_Training_001_seg_100,/home/shsingh/knowledge_distillation/knowledge...,HGG,HGG,HGG,HGG
4,BraTS20_Training_001,BraTS20_Training_001_seg_101,/home/shsingh/knowledge_distillation/knowledge...,HGG,HGG,HGG,HGG
...,...,...,...,...,...,...,...
1397,BraTS20_Training_010,BraTS20_Training_010_seg_10,/home/shsingh/knowledge_distillation/knowledge...,healthy,healthy,healthy,healthy
1398,BraTS20_Training_010,BraTS20_Training_010_seg_100,/home/shsingh/knowledge_distillation/knowledge...,HGG,HGG,HGG,HGG
1399,BraTS20_Training_010,BraTS20_Training_010_seg_101,/home/shsingh/knowledge_distillation/knowledge...,HGG,HGG,HGG,HGG
1400,BraTS20_Training_010,BraTS20_Training_010_seg_102,/home/shsingh/knowledge_distillation/knowledge...,HGG,HGG,HGG,HGG


In [20]:
x['subject_id'].value_counts()

BraTS20_Training_001    155
BraTS20_Training_002    155
BraTS20_Training_003    155
BraTS20_Training_004    155
BraTS20_Training_005    155
BraTS20_Training_006    155
BraTS20_Training_007    155
BraTS20_Training_008    155
BraTS20_Training_009    155
BraTS20_Training_010      7
Name: subject_id, dtype: int64

In [21]:
x['image_idx'].value_counts()

BraTS20_Training_001_seg_0      1
BraTS20_Training_007_seg_1      1
BraTS20_Training_007_seg_106    1
BraTS20_Training_007_seg_105    1
BraTS20_Training_007_seg_104    1
                               ..
BraTS20_Training_004_seg_1      1
BraTS20_Training_004_seg_0      1
BraTS20_Training_003_seg_99     1
BraTS20_Training_003_seg_98     1
BraTS20_Training_010_seg_103    1
Name: image_idx, Length: 1402, dtype: int64

In [22]:
x['image_idx'][0]

'BraTS20_Training_001_seg_0'